# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

d:\git\personal\udacity_aws_data_engineering\projects\1 - Data Modelling with Apache Cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster
*Note:* a Docker Compose file has been provided to easily spin-up a Cassandra cluster. Simply run `docker compose up -d`. On Windows, it is recommended to use Docker Desktop with the WSL backend.

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 
keyspace = 'project1'
try:
    query = f"""
    CREATE KEYSPACE IF NOT EXISTS {keyspace}
    WITH replication = {{'class': 'SimpleStrategy', 'replication_factor' : 1}};
    """
    session.execute(query)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace(keyspace=keyspace)
except Exception as e:
    print(e)

#### Set Pandas Row Factory

In [ ]:
# Define a custom row factory
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

# Set the row factory for Cassandra
session.row_factory = pandas_factory

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data
1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Query 1: Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [9]:
# Model table after query requirements
table_name = 'music_history_by_session';

drop_query = f'DROP TABLE IF EXISTS {table_name};'

query = f"""
CREATE TABLE IF NOT EXISTS {table_name} (
    sessionId INT,
    itemInSession INT,
    artist VARCHAR,
    song VARCHAR,
    length FLOAT,
    PRIMARY KEY (sessionId, itemInSession)
);
"""

try:
    session.execute(drop_query)
    session.execute(query)
    print(f"Table '{table_name}' created successfully.")
except Exception as e:
    print(e)
    print(f"Error during table creation: {e}")

Table 'music_history_by_session' created successfully.


In [10]:
# Insert data into the table
file = 'event_datafile_new.csv'

tmp = []
with open(file, encoding = 'utf8') as f:
    csvreader = csv.DictReader(f)
    next(csvreader) # skip header
    for row in csvreader:
        query = f"""
            INSERT INTO {table_name} (sessionId, itemInSession, artist, song, length) 
            VALUES (%s, %s, %s, %s, %s);
        """
        values = (int(row['sessionId']), int(row['itemInSession']), row['artist'], row['song'], float(row['length']))
        try:
            session.execute(query, values)
        except Exception as e:
            print(e)
            

In [26]:
# Verify the query results
table_name = 'music_history_by_session'
sessionId = 338
itemInSession = 4

query = f"""
SELECT artist, song, length
FROM {table_name}
WHERE sessionId = {sessionId}
AND itemInSession = {itemInSession}
"""

try:
    result_set = session.execute(query)
    df = result_set._current_rows
except Exception as e:
    print(e)

df.head()

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [36]:
# Model table after query requirements
table_name = 'music_history_by_user';

drop_query = f'DROP TABLE IF EXISTS {table_name};'

query = f"""
CREATE TABLE IF NOT EXISTS {table_name} (
    userId INT,
    sessionId INT,
    itemInSession INT,
    artist VARCHAR,
    song VARCHAR,
    user VARCHAR,
    PRIMARY KEY (userId, sessionId, itemInSession)
);
"""

try:
    session.execute(drop_query)
    session.execute(query)
    print(f"Table '{table_name}' created successfully.")
except Exception as e:
    print(e)
    print(f"Error during table creation: {e}")
                    

Table 'music_history_by_user' created successfully.


In [37]:
# Insert data into the table
file = 'event_datafile_new.csv'

tmp = []
with open(file, encoding = 'utf8') as f:
    csvreader = csv.DictReader(f)
    next(csvreader) # skip header
    for row in csvreader:
        query = f"""
            INSERT INTO {table_name} (userId, sessionId, itemInSession, artist, song, user) 
            VALUES (%s, %s, %s, %s, %s, %s);
        """
        values = (
            int(row['userId']),
            int(row['sessionId']),
            int(row['itemInSession']),
            row['artist'],
            row['song'],
            f"{row['firstName']} {row['lastName']}"
        )
        try:
            session.execute(query, values)
        except Exception as e:
            print(e)
            

In [38]:
# Verify the query results
table_name = 'music_history_by_user'
userId = 10
sessionId = 182

query = f"""
SELECT artist, song, user
FROM {table_name}
WHERE userId = {userId}
AND sessionId = {sessionId}
"""

try:
    result_set = session.execute(query)
    df = result_set._current_rows
except Exception as e:
    print(e)

df.head()

,artist,song,user
0,Down To The Bone,Keep On Keepin' On,Sylvie Cruz
1,Three Drives,Greece 2000,Sylvie Cruz
2,Sebastien Tellier,Kilometer,Sylvie Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie Cruz


In [ ]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


                    

### Drop the tables before closing out the sessions

In [4]:
## TO-DO: Drop the table before closing out the sessions

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()